# Self Query 

The code is here : [ai_chains/B_2_self_query.py](../python/ai_chains/B_2_self_query.py)  <br>
- Adapted from: https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/
- See also https://blog.langchain.dev/query-construction/

Can be executed in the WebApp 

In [ ]:
from dotenv import load_dotenv
from rich import print as rprint

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

In [ ]:
from src.ai_chains.B_2_self_query import get_query_constructor, get_retriever

In [ ]:
query_constructor = get_query_constructor({})
query = query_constructor.invoke(
    {"query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"}
)
debug(query)

In [ ]:
# retriever = get_retriever(config={"llm": None})
retriever = get_retriever(config={"llm": "gpt_35_azure"})
result = retriever.invoke("I want to watch a movie rated higher than 8.5")
debug(result)